# Filter wsn data

Load libraries

In [2]:
from wsn_client import query
import datetime, os
from matplotlib import pyplot
import numpy as np
import pandas as pd
import os.path


Load one station

In [3]:
# Time period
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019,6,1)#(2020, 10, 10)

# Load station
#station_name =   [ f"sw-{x:03d}"  for x in range(1,14) ]
#for name in station_name:
name = "sw-001"
#var_oi = ['tmp_temperature','bme_hum'] #[]'type','acc_x','acc_y','acc_z','ds1820','mb_distance','vl_distance','tmp_temperature'
df = query.query('postgresql', name=name, time__gte=start, time__lte=end, limit=2000000000000)

# Check if empty
if df.empty: 
    print(name," --- Frame is >> EMPTY <<")
else:     
    # Remove non recorded data
    if 'type' in df.columns:
        df = df[df.type==0]
    print(name)
    # Remove duplicated indexes and frames
    df=df.drop_duplicates(['time', 'frame'])
    # Printing Number of columns and names
    print('Number of columns :', df.shape[1]) 
    print(df.columns)

sw-001
Number of columns : 29
Index(['time', 'frame', 'type', 'bat', 'acc_x', 'acc_y', 'acc_z', 'bme_tc',
       'int_tc', 'bme_hum', 'int_hum', 'bme_pres', 'int_pres', 'mb_distance',
       'vl_distance', 'tmp_temperature', 'ds1820', 'altitude', 'latitude',
       'longitude', 'gps_accuracy', 'gps_satellites', 'mlx_object',
       'mlx_ambient', 'received', 'momsn', 'iridium_cep', 'iridium_latitude',
       'iridium_longitude'],
      dtype='object')


In [21]:
# print("NaN percentage per column")
# print((df.isna().sum()/len(df)*100).round(2))

In [4]:
sensor_temp = ['tmp_temperature','bme_tc']#, 'ds1820', 'mlx_ambient'
sensor_hum = ['bme_hum',]
sensor_wind = ['wind_dir','wind_gust','wind_speed']
sensor_snow = ['mb_distance','vl_distance']
sensor_pres = ['bme_pres']
sensor_acc = ['acc_x','acc_y','acc_z']

In [5]:
coord=df[df.columns[df.columns.isin(['latitude','longitude','altitude'])]][df.gps_satellites>7].agg(['mean','median', 'std']).round(4)

In [6]:
# Extract wanted and existing columns
coltoget2 = ['bme_tc','bme_hum', 'bme_pres', 'mlx_object','mb_distance',]
df[df.columns[df.columns.isin(coltoget2)]]

,bme_tc,bme_hum,bme_pres,mb_distance,mlx_object
time,,,,,
2019-04-06 15:00:00,22.820000,12.080078,103088.164062,"[2161, 2164, 2166, 2165, 2165]",NaN
2019-04-06 15:10:00,23.020000,11.994141,103092.312500,"[1203, 1200, 1201, 1207, 1210]",NaN
2019-04-06 15:20:00,23.590000,10.735352,103091.242188,"[1286, 1913, 1910, 1910, 1911]",NaN
2019-04-06 15:30:00,23.889999,11.067383,103101.437500,"[1236, 1247, 1234, 1247, 1237]",NaN
2019-04-06 15:40:00,24.070000,10.757812,103115.226562,"[1303, 1254, 1251, 1221, 1929]",NaN
...,...,...,...,...,...
2019-05-31 21:20:00,-0.140000,65.060547,96785.093750,"[1087, 1088, 1088, 1088, 1090]",-4.929993
2019-05-31 21:30:00,-0.780000,67.336914,96802.757812,"[1087, 1086, 1087, 1089, 1088]",-5.109985
2019-05-31 21:40:00,-0.640000,66.371094,96792.992188,"[1088, 1086, 1089, 1089, 1086]",-4.890015


In [7]:
type(df)

pandas.core.frame.DataFrame

In [9]:
def quality_control_air_temperature(df,f_update=False,f_print=True):
    """Apply quality control on air temperature data.

    The following tests are applied:
        - Must pass monthly temperature limit. QC.

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        data downloaded from wsn
    flag_update : boolean
        Flag to recompute the monthly min and max of the temperature (saved 
        in csv at data_qc/temperature_,monthly_limit.csv)
    """

    sensor_temp = ['tmp_temperature','bme_tc']
        
    
    if all(~df.columns.isin(sensor_temp)):
        if f_print: print('No temperature data defined as:')
        if f_print: print(sensor_temp) 
        temp=None
        return(temp)
        
    # Update monthly limits and save it in a csv file (can be manually edited)
    filename='data_qc/temperature_monthly_limit.csv'
    if (f_update) | (~os.path.isfile(filename)): 
        update_file_temperature_monthly_limits(df.tmp_temperature,filename,f_print)
    
    # Load and select data source
    if f_print: print('Load temperature data')        
    col_to_get=df.columns.isin([sensor_temp[0]])
    if any(col_to_get):
        temp=df[df.columns[col_to_get]]
        if f_print: print(temp)
        
        # Quality Control -- 0) -- Remove installation period
        
        
        # Quality Control -- 1) -- Monthly limit
        limit_table=pd.read_csv(filename)
        temp.loc[:,sensor_temp[0] + '_qc']=df[[sensor_temp[0]]].apply(lambda x: qc_temperature_monthly_limits(x,limit_table,
                                                                                                      var='temp'),axis=1)
        
        # Quality Control -- 2) -- Rnage limit
        if f_print: print(temp)
    
    return(temp)



def update_file_temperature_monthly_limits(df,filename='data_qc/temperature_monthly_limit.csv',f_print=True):
    """Compute monthly air temperature minum and maximum.

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        temperature data downloaded from wsn
    """
    if f_print: print('Update monthly limits for Air Temperature')
    monthly_data=df.groupby(df.index.month).agg(['min', 'max','std','count']).round(2)
    monthly_data=monthly_data.rename(columns = {'min':'temp_min','max':'temp_max',
                                                'std':'temp_std','count':'temp_count'})
    monthly_data.to_csv(filename,index_label='month')
    
    
def qc_temperature_monthly_limits(x,limit_table, var='temp'):
    """Compute whether points are in range and assign quality assessment.
    
    The function is built for apply() and needs to be run as followed:
    qc_range=df[['tmp_temperature']].apply(lambda x: fun(x,limit_table),axis=1)
    
    Parameters
    ----------
    x : pandas.core.frame.DataFrame (one column)
        temperature data downloaded from wsn
    limit_table : pandas.core.frame.DataFrame 
        defining the limit in temperature range per month
    var : string
        name of the variable
    """
    limit=limit_table[limit_table.month.values==x.name.month]
    out=(x<limit.iloc[0][var + '_min']) | (x>limit.iloc[0][var + '_max'])
    out.at[out]='QC1-1'
    return out

def qc_sensor_range_limits(x,limit_table, var='temp'):
    """Logical selection of points outside of the sensor range 
    and allocation of quality assessment.
    
    The function is built for apply() and needs to be run as followed:
    qc_range=df[['tmp_temperature']].apply(lambda x: fun(x,limit_table),axis=1)
    
    Parameters
    ----------
    x : pandas.core.frame.DataFrame (one column)
        temperature data downloaded from wsn
    limit_table : pandas.core.frame.DataFrame 
        defining the limit in temperature range per month
    var : string
        name of the variable
    """
    limit=limit_table
    out=(x<limit.iloc[0][var + '_min']) | (x>limit.iloc[0][var + '_max'])
    out.at[out]='QC1-0'
    return out

## Function to update monthly limits in temperature

In [21]:
df=df[0:1000]

In [22]:
sensor_temp = ['tmp_temperature','bme_tc']
# Update monthly limits and save it in a csv file (can be manually edited)
filename='data_qc/temperature_monthly_limit.csv'

# Load and select data source
print('Load temperature data')        
col_to_get=df.columns.isin([sensor_temp[0]])
if any(col_to_get):
    temp=df[df.columns[col_to_get]]
#     print(temp)

    # Quality Control -- 1) -- Monthly limit
    limit_table=pd.read_csv(filename)
    df2=pd.DataFrame()
    
    tm=df[[sensor_temp[0]]].apply(lambda x: qc_temperature_monthly_limits(x,limit_table,var='temp'))
    df2[sensor_temp[0] + '_qc']=tm
    print(temp)


Load temperature data


AttributeError: 'str' object has no attribute 'month'

In [28]:
df2


""


## TEST ZONE

In [17]:
tmp=quality_control_air_temperature(df)

Update monthly limits for Air Temperature
Load temperature data
                     tmp_temperature
time                                
2019-04-06 15:00:00          22.6875
2019-04-06 15:10:00          22.8750
2019-04-06 15:20:00          23.4375
2019-04-06 15:30:00          23.7500
2019-04-06 15:40:00          23.9375
...                              ...
2019-04-13 13:10:00          -6.5625
2019-04-13 13:20:00          -6.3750
2019-04-13 13:30:00          -6.6250
2019-04-13 13:40:00          -6.6875
2019-04-13 13:50:00          -6.6250

[1000 rows x 1 columns]
                     tmp_temperature  tmp_temperature_qc
time                                                    
2019-04-06 15:00:00          22.6875                 NaN
2019-04-06 15:10:00          22.8750                 NaN
2019-04-06 15:20:00          23.4375                 NaN
2019-04-06 15:30:00          23.7500                 NaN
2019-04-06 15:40:00          23.9375                 NaN
...                            

/Users/pmlefeuv/miniconda3/envs/uioData/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/pmlefeuv/miniconda3/envs/uioData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [390]:
# tmp=quality_control_air_temperature(df[df.columns[1:3]])

In [9]:
tmp

,tmp_temperature,tmp_temperature_qc
time,,
2019-04-06 15:00:00,22.6875,False
2019-04-06 15:10:00,22.8750,False
2019-04-06 15:20:00,23.4375,False
2019-04-06 15:30:00,23.7500,False
2019-04-06 15:40:00,23.9375,False
...,...,...
2020-01-07 19:00:00,-8.6250,False
2020-01-07 19:10:00,-8.6875,False
2020-01-07 19:20:00,-8.5625,False


In [23]:
df.columns[df.columns.isin(sensor_temp)]

Index(['bme_tc', 'tmp_temperature'], dtype='object')

In [24]:
sensor_temp[0] + '_qc'

'tmp_temperature_qc'

## Function to control periods of station installation or maintenance

In [25]:
(df.index>pd.to_datetime('2019-04-08')) & (df.index<pd.to_datetime('2019-04-10'))

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [28]:
df.all()
df.columns[df.columns.isin(sensor_temp)]

Index(['bme_tc', 'tmp_temperature'], dtype='object')